In [1]:
from google.colab import drive
import nbformat, os

drive.mount('/content/drive', force_remount=True)
SRC = '/content/drive/MyDrive/Colab Notebooks/Untitled20.ipynb'
DST = '/content/drive/MyDrive/colab_saves/LoRA for prediction.ipynb'

with open(SRC, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)


nb.metadata.pop('widgets', None)
for cell in nb.cells:
    cell.metadata.pop('widgets', None)
os.makedirs(os.path.dirname(DST), exist_ok=True)
with open(DST, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

print("✅ Clean copy saved to", DST)

KeyboardInterrupt: 

In [1]:
import os, math, numpy as np, pandas as pd, torch, torch.nn as nn
from sklearn.model_selection import train_test_split
from google.colab import drive

drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/colab_saves/byday_with_diff_clean.pkl"
df = pd.read_pickle(file_path)
df = df[['titles', 'difference']].dropna()
df = df[df['titles'].astype(str).str.len() > 0]
df = df.drop_duplicates(subset=['titles']).reset_index(drop=True)

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
len(train_df), len(val_df), train_df.head(2)
train_df = train_df.reset_index(drop=True)
val_df   = val_df.reset_index(drop=True)

Mounted at /content/drive


In [2]:
from transformers import AutoTokenizer, AutoModel
import torch

MODEL_NAME = "Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

dtype = torch.float16 if torch.cuda.is_available() else torch.float32
base_model = AutoModel.from_pretrained(MODEL_NAME, torch_dtype=dtype)


if hasattr(base_model.config, "use_cache"):
    base_model.config.use_cache = False
hidden_size = base_model.config.hidden_size




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

In [3]:

import torch, torch.nn as nn

def mean_pool(last_hidden_state, attention_mask):
    mask = attention_mask.unsqueeze(-1).expand_as(last_hidden_state).float()
    summed = (last_hidden_state * mask).sum(dim=1)
    counts = torch.clamp(mask.sum(dim=1), min=1e-9)
    return summed / counts

class EncoderWithRegHead(nn.Module):
    def __init__(self, encoder, hidden_size):
        super().__init__()
        self.encoder  = encoder
        self.reg_head = nn.Linear(hidden_size, 1)

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        out = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )
        last_hidden = out.last_hidden_state
        pooled = mean_pool(last_hidden, attention_mask)
        pred = self.reg_head(pooled).squeeze(-1)
        loss = None
        if labels is not None:
            loss = nn.MSELoss()(pred, labels)
        return {"loss": loss, "logits": pred}


model_core = EncoderWithRegHead(base_model, base_model.config.hidden_size)

In [4]:
from torch.utils.data import Dataset
import torch
import pandas as pd

class TitlesRegDataset(Dataset):

    def __init__(self, df: pd.DataFrame, tokenizer, max_len: int = 64, text_col: str = "titles", y_col: str = "difference"):
        df = df[[text_col, y_col]].copy()
        df[y_col] = pd.to_numeric(df[y_col], errors="coerce")
        df = df.dropna(subset=[y_col]).reset_index(drop=True)

        self.df = df
        self.tk = tokenizer
        self.max_len = max_len
        self.text_col = text_col
        self.y_col = y_col

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        text = str(row[self.text_col])
        y    = float(row[self.y_col])

        enc = self.tk(
            text,
            truncation=True,
            padding=False,
            max_length=self.max_len,
            return_tensors="pt"
        )
        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = torch.tensor(y, dtype=torch.float)
        return item


In [5]:
train_ds = TitlesRegDataset(train_df, tokenizer, max_len=64)
val_ds   = TitlesRegDataset(val_df, tokenizer, max_len=64)

In [6]:

from peft import LoraConfig, get_peft_model, TaskType

lora_cfg = LoraConfig(
    task_type=TaskType.FEATURE_EXTRACTION,
    r=2,
    lora_alpha=8,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)
model_core.encoder = get_peft_model(model_core.encoder, lora_cfg)


if hasattr(model_core.encoder.config, "use_cache"):
    model_core.encoder.config.use_cache = False
if hasattr(model_core.encoder, "gradient_checkpointing_enable"):
    model_core.encoder.gradient_checkpointing_enable()
    if hasattr(model_core.encoder, "enable_input_require_grads"):
        model_core.encoder.enable_input_require_grads()
)
with torch.no_grad():
    enc = tokenizer("test", return_tensors="pt")
    out = model_core.encoder(**enc, return_dict=True)
    print("Has last_hidden_state:", hasattr(out, "last_hidden_state"))



Has last_hidden_state: True


In [7]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [9]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00


In [10]:
import evaluate
mse_metric = evaluate.load("mse")
mae_metric = evaluate.load("mae")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = preds.reshape(-1)
    labels = labels.reshape(-1)
    mse = mse_metric.compute(predictions=preds, references=labels)["mse"]
    mae = mae_metric.compute(predictions=preds, references=labels)["mae"]
    rmse = np.sqrt(mse)
    ybar = labels.mean()
    ss_res = float(((labels - preds) ** 2).sum())
    ss_tot = float(((labels - ybar) ** 2).sum()) + 1e-12
    r2 = 1 - ss_res / ss_tot
    return {"rmse": rmse, "mae": mae, "r2": r2}



In [11]:
from transformers import TrainingArguments
MAX_LEN = 48

from transformers import TrainingArguments
import torch

training_args = TrainingArguments(
    output_dir="./lora_news_reg",
    learning_rate=8e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    weight_decay=0.0,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,


    dataloader_num_workers=0,
    dataloader_pin_memory=False,
    remove_unused_columns=False,

    gradient_checkpointing=False,
    fp16=torch.cuda.is_available(),
    report_to="none",
    logging_steps=25,
    max_steps=200,
)





In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model_core,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

import torch
with torch.no_grad():
    metrics = trainer.evaluate()
print(metrics)



/tmp/ipython-input-515510732.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
25,6.799100
50,12.278500
75,9.927400
100,5.940100
125,10.121900
150,7.969000
175,9.592100
200,6.244400


{'eval_loss': 40.75928497314453, 'eval_rmse': 6.384300544828414, 'eval_mae': 3.5156502577547846, 'eval_r2': 0.009425254909084635, 'eval_runtime': 80.1368, 'eval_samples_per_second': 1.298, 'eval_steps_per_second': 0.649, 'epoch': 1.9297820823244551}
